In [154]:
# You need to replace these placeholders with your keys

consumer_key = 'CONSUMER_KEY_PLACEHOLDER'
consumer_secret = 'CONSUMER_SECRET_PLACEHOLDER'
access_token = 'ACCESS_TOKEN_PLACEHOLDER'
access_secret = 'ACCESS_SECRET_PLACEHOLDER'

In [155]:
# importing libraries
import pandas as pd
import requests
import tweepy
import numpy as np
import time
import json
import os
import contextlib

## Gathering data
---

In [156]:
df_archive = pd.read_csv('twitter-archive-enhanced.csv')
df_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [157]:
df_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [158]:
# retrieve image predictions
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"

# request from the url
req = requests.get(url)
if req.status_code == requests.codes.ok:
    req.encoding = 'utf-u'
    with open('image_predictions.txt', 'w') as infile:
        # write the text into the txt file
        infile.write(req.text)
        
# read the content using pandas 
df_image_predictions = pd.read_csv("image_predictions.txt", sep= '\t')

df_image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [159]:
df_image_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [160]:
# get the tweet IDs from df_archive and df_image_predictions
tweet_IDs = np.array(df_archive['tweet_id'].values, dtype = str)
tweet_IDs = np.concatenate((np.array(df_image_predictions['tweet_id'].values, dtype = str), tweet_IDs))
tweet_IDs = np.unique(tweet_IDs)
tweet_IDs

array(['666020888022790149', '666029285002620928', '666033412701032449',
       ..., '891815181378084864', '892177421306343426',
       '892420643555336193'],
      dtype='<U21')

In [161]:
#create the api instance
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, 
                 parser = tweepy.parsers.JSONParser(),
                 wait_on_rate_limit = True,
                 wait_on_rate_limit_notify = True)

# create the lists where we will store the results from the twitter api
tweets = []
errors = []

for tweet_id in tweet_IDs:
    start = time.time()
    try:
        result = api.get_status(tweet_id, tweet_mode = 'extended')
        # Uncomment the lines below to see a JSON dump of one result
        # print(json.dumps(result, indent = 2))
        # break
        
        favorites = result['favorite_count']
        retweets = result['retweet_count']
        
        tweets.append({'tweet_id': int(tweet_id),
                        'favorites': int(favorites),
                        'retweets': int(retweets)})
    
    except Exception as e:
        errors.append(tweet_id)
    end = time.time()
    request_time = end - start
    print("time required: ", request_time)

time required:  0.339216947555542
time required:  0.40778326988220215
time required:  0.27196478843688965
time required:  0.3075380325317383
time required:  0.25479769706726074
time required:  0.28074002265930176
time required:  0.2666308879852295
time required:  0.3009469509124756
time required:  0.2993500232696533
time required:  0.267535924911499
time required:  0.2920517921447754
time required:  0.26284313201904297
time required:  0.28342509269714355
time required:  0.2632472515106201
time required:  0.28727173805236816
time required:  0.27858614921569824
time required:  0.2761249542236328
time required:  0.2888009548187256
time required:  0.27950406074523926
time required:  0.2603418827056885
time required:  0.2966742515563965
time required:  0.2953498363494873
time required:  0.3507049083709717
time required:  0.2901031970977783
time required:  0.26840996742248535
time required:  0.2953050136566162
time required:  0.29866695404052734
time required:  0.2823150157928467
time requir

time required:  0.2730369567871094
time required:  0.2532980442047119
time required:  0.28824901580810547
time required:  0.2703988552093506
time required:  0.26407289505004883
time required:  0.2884101867675781
time required:  0.3151261806488037
time required:  0.2811892032623291
time required:  0.29926276206970215
time required:  0.2836177349090576
time required:  0.29942989349365234
time required:  0.25588297843933105
time required:  0.28612208366394043
time required:  0.26669788360595703
time required:  0.28611278533935547
time required:  0.26100873947143555
time required:  0.27952003479003906
time required:  0.26387500762939453
time required:  0.2678399085998535
time required:  0.2864878177642822
time required:  0.2665739059448242
time required:  0.3006911277770996
time required:  0.258394718170166
time required:  0.25830888748168945
time required:  0.284898042678833
time required:  0.2603449821472168
time required:  0.2650299072265625
time required:  0.25342297554016113
time requ

time required:  0.25858616828918457
time required:  0.26099371910095215
time required:  0.27046704292297363
time required:  0.27952098846435547
time required:  0.27043628692626953
time required:  0.264970064163208
time required:  0.255856990814209
time required:  0.30455803871154785
time required:  0.2768268585205078
time required:  0.27171897888183594
time required:  0.27556300163269043
time required:  0.2877769470214844
time required:  0.307009220123291
time required:  0.27444982528686523
time required:  0.33618664741516113
time required:  0.2764420509338379
time required:  0.2941300868988037
time required:  0.2714869976043701
time required:  0.26424217224121094
time required:  0.257213830947876
time required:  0.32147789001464844
time required:  0.2669999599456787
time required:  0.25994396209716797
time required:  0.2967259883880615
time required:  0.2736039161682129
time required:  0.2707831859588623
time required:  0.28214383125305176
time required:  0.287398099899292
time requir

time required:  0.36098194122314453
time required:  0.32709288597106934
time required:  0.35697197914123535
time required:  0.29025983810424805
time required:  0.3537888526916504
time required:  0.30965614318847656
time required:  0.2730722427368164
time required:  0.25600099563598633
time required:  0.2655320167541504
time required:  0.2863962650299072
time required:  0.40533018112182617
time required:  0.2776150703430176
time required:  0.286837100982666
time required:  0.27657485008239746
time required:  0.2724010944366455
time required:  0.2816920280456543
time required:  0.34648990631103516
time required:  0.28757810592651367
time required:  0.2641279697418213
time required:  0.25458717346191406
time required:  0.26787590980529785
time required:  0.2744169235229492
time required:  0.2917208671569824
time required:  0.30016183853149414
time required:  0.34806394577026367
time required:  0.2597191333770752
time required:  0.26824307441711426
time required:  0.31118321418762207
time 

time required:  0.267179012298584
time required:  0.3040299415588379
time required:  0.268308162689209
time required:  0.31038999557495117
time required:  0.35796189308166504
time required:  0.35667872428894043
time required:  0.2713761329650879
time required:  0.34277987480163574
time required:  0.29149794578552246
time required:  0.3596072196960449
time required:  0.28853702545166016
time required:  0.27977585792541504
time required:  0.2742938995361328
time required:  0.3020930290222168
time required:  0.2740509510040283
time required:  0.2652459144592285
time required:  0.2784860134124756
time required:  0.2847421169281006
time required:  0.26525282859802246
time required:  0.253173828125
time required:  0.2626001834869385
time required:  0.28647613525390625
time required:  0.31120991706848145
time required:  0.2677450180053711
time required:  0.28747081756591797
time required:  0.2626183032989502
time required:  0.2547128200531006
time required:  0.2823209762573242
time required: 

...

time required:  0.2799949645996094
time required:  0.27462077140808105
time required:  0.28492307662963867
time required:  0.26877689361572266
time required:  0.2611410617828369
time required:  0.29853320121765137
time required:  0.2718188762664795
time required:  0.33222413063049316
time required:  0.28621602058410645
time required:  0.30577921867370605
time required:  0.3386688232421875
time required:  0.2666029930114746
time required:  0.3044757843017578
time required:  0.2699718475341797
time required:  0.28275132179260254
time required:  0.27895283699035645
time required:  0.2879810333251953
time required:  0.3277902603149414
time required:  0.2566659450531006
time required:  0.2793426513671875
time required:  0.31368589401245117
time required:  0.3103780746459961
time required:  0.2541210651397705
time required:  0.2960331439971924
time required:  0.2548210620880127
time required:  0.2774379253387451
time required:  0.26331496238708496
time required:  0.284060001373291
time requi

time required:  0.3310079574584961
time required:  0.32042789459228516
time required:  0.4259958267211914
time required:  0.3160557746887207
time required:  0.26155900955200195
time required:  0.30186986923217773
time required:  0.2966651916503906
time required:  0.3026001453399658
time required:  0.27729201316833496
time required:  0.2747931480407715
time required:  0.3133108615875244
time required:  0.29151010513305664
time required:  0.43491291999816895
time required:  0.2971041202545166
time required:  0.2547121047973633
time required:  0.2719149589538574
time required:  0.2565789222717285
time required:  0.25295066833496094
time required:  0.2522733211517334
time required:  0.3082001209259033
time required:  0.29667186737060547
time required:  0.26136279106140137
time required:  0.3245513439178467
time required:  0.28154778480529785
time required:  0.2886049747467041
time required:  0.2710130214691162
time required:  0.6804561614990234
time required:  0.2835240364074707
time requi

time required:  0.285297155380249
time required:  0.2599611282348633
time required:  0.27056097984313965
time required:  0.27951526641845703
time required:  0.27332496643066406
time required:  0.30310511589050293
time required:  0.3461430072784424
time required:  0.2911391258239746
time required:  0.3062567710876465
time required:  0.2718651294708252
time required:  0.2613718509674072
time required:  0.25434088706970215
time required:  0.2864110469818115
time required:  0.2640671730041504
time required:  0.3043241500854492
time required:  0.31357789039611816
time required:  0.2798020839691162
time required:  0.2772543430328369
time required:  0.2824709415435791
time required:  0.2674369812011719
time required:  0.28066515922546387
time required:  0.2942211627960205
time required:  0.27063894271850586
time required:  0.2753019332885742
time required:  0.2803831100463867
time required:  0.329679012298584
time required:  0.26552605628967285
time required:  0.3415040969848633
time required

time required:  0.25649499893188477
time required:  0.28782200813293457
time required:  0.26368069648742676
time required:  0.3011629581451416
time required:  0.25841212272644043
time required:  0.2826259136199951
time required:  0.28188419342041016
time required:  0.27047085762023926
time required:  0.335172176361084
time required:  0.29479193687438965
time required:  0.2907369136810303
time required:  0.2634928226470947
time required:  0.28353381156921387
time required:  0.2686648368835449
time required:  0.31581783294677734
time required:  0.3268861770629883
time required:  0.2725701332092285
time required:  0.3177459239959717
time required:  0.3197059631347656
time required:  0.2614479064941406
time required:  0.2640798091888428
time required:  0.2860848903656006
time required:  0.29260706901550293
time required:  0.260390043258667
time required:  0.2912609577178955
time required:  0.3088080883026123
time required:  0.25701379776000977
time required:  0.2780938148498535
time requir

time required:  0.39117002487182617
time required:  0.35611605644226074
time required:  0.2845790386199951
time required:  0.4845468997955322
time required:  0.7144699096679688
time required:  0.3097190856933594
time required:  0.3176119327545166
time required:  0.29578614234924316
time required:  0.2569911479949951
time required:  0.27400994300842285
time required:  0.2954292297363281
time required:  0.3439497947692871
time required:  0.33528685569763184
time required:  0.2567739486694336
time required:  0.30280327796936035
time required:  0.2720019817352295
time required:  0.33625292778015137
time required:  0.23769688606262207
time required:  0.2607309818267822
time required:  0.2857069969177246
time required:  0.3065328598022461
time required:  0.27431297302246094
time required:  0.28047800064086914
time required:  0.27828192710876465
time required:  0.2975430488586426
time required:  0.28058910369873047
time required:  0.3627290725708008
time required:  0.2657592296600342
time req

In [162]:
print("The number of results from twitter", len(tweets))
print("The number of errors", len(errors))

The number of results from twitter 2348
The number of errors 8


In [163]:
with contextlib.suppress(FileNotFoundError):
    os.remove('tweet_json.txt')

df_json_tweets = pd.DataFrame(tweets, columns = ['tweet_id', 'favorites', 'retweets'])

df_json_tweets.to_csv('tweet_json.txt', encoding = 'utf-8', index=False)
df_json_tweets.head()

,tweet_id,favorites,retweets
0,666020888022790149,2526,523
1,666029285002620928,131,47
2,666033412701032449,126,46
3,666044226329800704,305,143
4,666049248165822465,110,40


## Assesing data
---

### Visual inspection

In [164]:
df_archive

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [165]:
df_image_predictions

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


Observation: p1_conf > p2conf > p3_conf. Therefor it is not needed to keep p2, p2_conf, p2_dog, p3, p3_conf, p3_dog
Observation 2: Some pictures are not of dogs i.e. https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg identified by the neural network as hen shows indeed a hen.
Obervation 3: Some picture identified as not a dog are in fact dogs i.e. https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg identified as a shopping cart shows a dog sitting in a shopping cart.
        

In [167]:
df_json_tweets

,tweet_id,favorites,retweets
0,666020888022790149,2526,523
1,666029285002620928,131,47
2,666033412701032449,126,46
3,666044226329800704,305,143
4,666049248165822465,110,40
5,666050758794694657,135,59
6,666051853826850816,1243,869
7,666055525042405380,447,259
8,666057090499244032,303,144
9,666058600524156928,115,59


I do not notice anything remarkable in this table. 

### Programmatic inspection

In [168]:
df_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

Observation: many variables have the wrong type i.e. object should be string for name source and text. 

In [169]:
df_archive.nunique()

tweet_id                      2356
in_reply_to_status_id           77
in_reply_to_user_id             31
timestamp                     2356
source                           4
text                          2356
retweeted_status_id            181
retweeted_status_user_id        25
retweeted_status_timestamp     181
expanded_urls                 2218
rating_numerator                40
rating_denominator              18
name                           957
doggo                            2
floofer                          2
pupper                           2
puppo                            2
dtype: int64

In [170]:
df_archive.source.unique()

array([ '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
       '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
       '<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>',
       '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>'], dtype=object)

Observation: Dirty. As observed before the sources are hard to read. As there only 4 different sources this is easy to fix with search and replace.

In [171]:
df_archive.doggo.unique()

array(['None', 'doggo'], dtype=object)

Observation: This is a tidyness issue doggo, floofer, pupper, puppo should be in one variabele.

In [172]:
df_archive['name'].value_counts()

None          745
a              55
Charlie        12
Lucy           11
Oliver         11
Cooper         11
Tucker         10
Penny          10
Lola           10
Winston         9
Bo              9
the             8
Sadie           8
Buddy           7
Toby            7
Daisy           7
Bailey          7
an              7
Rusty           6
Leo             6
Bella           6
Jax             6
Jack            6
Scout           6
Dave            6
Oscar           6
Milo            6
Stanley         6
Koda            6
Finn            5
             ... 
Coopson         1
Ralf            1
General         1
Lorelei         1
Perry           1
Grizzwald       1
Gilbert         1
Clybe           1
O               1
Chloe           1
Alexander       1
Stuart          1
Flash           1
Gabby           1
Sweets          1
Cora            1
Ronnie          1
Corey           1
life            1
Binky           1
Puff            1
Beemo           1
Bloop           1
Erik            1
Taz       

This is a dirty data issue. Names like "None", "a" , "an", and "the" are not likely dog names. 

In [173]:
df_archive['rating_numerator'].value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

Observation: As mentioned in the assignment many numerators are above 10. This is not an issue.

In [174]:
df_archive['rating_denominator'].value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

Observation: Most keep the denominator on 10. A few choose another scale. Hmm I don't trust this. Time for an extra visual inspection.

In [175]:
df_not_ten = df_archive[df_archive['rating_denominator'] != 10]
df_not_ten[['tweet_id', 'text','rating_numerator','rating_denominator']]

,tweet_id,text,rating_numerator,rating_denominator
313,835246439529840640,@jonnysun @Lin_Manuel ok jomny I know you're e...,960,0
342,832088576586297345,@docmisterio account started on 11/15/15,11,15
433,820690176645140481,The floofs have been released I repeat the flo...,84,70
516,810984652412424192,Meet Sam. She smiles 24/7 &amp; secretly aspir...,24,7
784,775096608509886464,"RT @dog_rates: After so many requests, this is...",9,11
902,758467244762497024,Why does this never happen at my front door......,165,150
1068,740373189193256964,"After so many requests, this is Bretagne. She ...",9,11
1120,731156023742988288,Say hello to this unbelievably well behaved sq...,204,170
1165,722974582966214656,Happy 4/20 from the squad! 13/10 for all https...,4,20
1202,716439118184652801,This is Bluebert. He just saw that both #Final...,50,50


Observation: when the denominator is not 10 it is often caused by other fraction being misidentified as the rating i.e 7/11 the store or 3 1/2 legged or 11/15/15 the date.

In [176]:
df_image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


Observation: The 2074 records found here is below the records found for the df_archive dataset. Meaning that there are tweets without an image.

In [177]:
df_image_predictions.nunique()

tweet_id    2075
jpg_url     2009
img_num        4
p1           378
p1_conf     2006
p1_dog         2
p2           405
p2_conf     2004
p2_dog         2
p3           408
p3_conf     2006
p3_dog         2
dtype: int64

In [178]:
df_image_predictions['jpg_url'].value_counts()

https://pbs.twimg.com/media/CkNjahBXAAQ2kWo.jpg                                            2
https://pbs.twimg.com/media/CdHwZd0VIAA4792.jpg                                            2
https://pbs.twimg.com/media/ChK1tdBWwAQ1flD.jpg                                            2
https://pbs.twimg.com/media/C12x-JTVIAAzdfl.jpg                                            2
https://pbs.twimg.com/media/CV_cnjHWUAADc-c.jpg                                            2
https://pbs.twimg.com/media/CsGnz64WYAEIDHJ.jpg                                            2
https://pbs.twimg.com/ext_tw_video_thumb/815965888126062592/pu/img/JleSw4wRhgKDWQj5.jpg    2
https://pbs.twimg.com/media/CiibOMzUYAA9Mxz.jpg                                            2
https://pbs.twimg.com/media/CeRoBaxWEAABi0X.jpg                                            2
https://pbs.twimg.com/media/CWza7kpWcAAdYLc.jpg                                            2
https://pbs.twimg.com/ext_tw_video_thumb/817423809049493505/pu/img/5OF

In [179]:
df_json_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2348 entries, 0 to 2347
Data columns (total 3 columns):
tweet_id     2348 non-null int64
favorites    2348 non-null int64
retweets     2348 non-null int64
dtypes: int64(3)
memory usage: 55.1 KB


In [180]:
# this returns the number of unique values in each column
df_json_tweets.nunique()

tweet_id     2348
favorites    2002
retweets     1739
dtype: int64

### Tidiness issues (structual issues)
* To much information in df_image_predictions dataframe, (tweet_id and jpg_url is what we need)
* Various stages of dogs (doggo, floofer, pupper, puppo) are in four columns.
* All dataframes should be merged into one.

### Dirtiness issues (quality)
* wrong datatypes for seeral columns
* Tweets without images
* Dataset contains retweets
* Incorrect dog names
* Sources difficult to read
* Some tweet_ids have the same jpg_url
* Some pictures are not of dogs
* Some pictures are flagged as not a dog but are in fact of a dog.
* Fractions in the text are mistaken for rates.